<a href="https://colab.research.google.com/github/JakeOh/202505_BD50/blob/main/lab_da/ml21_llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLM(Large Language Model, 거대 언어 모델)

*   시퀀스-시퀀스 작업(Sequence-to-Sequence)
    *   시퀀스 데이터를 입력받아서 시퀀스 데이터를 출력하는 작업
    *   자연어 처리(NLP, Natural Language Processing) 분야에서 요약, 번역 등의 작업 해당
    *   두 개의 (순환) 신경망을 연결한 인코더-디코더(encoder-decoder) 구조가 널리 사용
*   어텐션 메커니즘(Attention mechanism)
    *   인코더-디코더 구조에서 사용된 순환 신경망의 성능을 향상시키기 위해서 고안
    *   기존에는 인코더의 마지막 타임스텝에서 출력한 은닉 상태만을 사용해서 디코더가 새로운 텍스트를 생성
    *   어텐션 메커니즘은 모든 타임스텝에서 인코더가 출력한 은닉 상태를 디코더가 참조할 수 있도록 고안
    *   디코더가 새로운 토큰을 생성할 때 인코더가 처리한 토큰들 중에서 어떤 토큰에 주의(attention)를 기울일 지를 결정. (입력 토큰들마다 디코더가 중요도를 다르게 부여)
*   트랜스포머 모델(Transformer model)
    *   어텐션 메커니즘을 기반으로 하여 인코더-디코더 구조에서 순환층을 제거
    *   인코더에서 한 번에 하나의 토크씩 처리하지 않고 입력 테스트 전체를 한 번에 처리
    *   핵심 구성 요소
        *   멀티 헤드 어텐션(multi-head attention)
        *   층 정규화(layer normalization)
        *   잔차 연결(residual connection)
        *   피드 포워드 네트워크(feed-forward network)

<img src="https://pbs.twimg.com/media/GJg3QtMXwAAvUT4?format=jpg&name=large"
    alt="LLM 가계도" />

*   Encoder 기반 모델
    *   텍스트 (긍정/부정) 분류
    *   개체명 인식 - 텍스트에서 사람 이름, 지역 이름, 회사 이름 등의 고유 명사를 식별.
    *   BERT, RoBERTa, ...
*   Encoder-Decoder 기반 모델
    *   문서 요약, 번역, 질문-답변(질문과 문맥 텍스트가 주어졌을 때 문맥 안에서 답을 찾아서 생성)
    *   T5, BART, ...
*   Decoder 기반 모델
    *   텍스트 생성 - 챗봇, 질문 답변, 요약, 번역
    *   디코더:
        *   이전까지 생성한 텍스트를 입력받아 다음 토큰을 예측하는 방식
        *   인코더로부터의 입력이 없으면 디코더는 아무것도 생성할 수 없음.
        *   이전에 생성한 텍스트인 것처럼 어떤 텍스트를 입력해 주면 인코더 도움 없이 다음 토큰을 예측.
        *   프롬프트(prompt): 이전에 생성한 텍스트인 것처럼 전달하는 초기 텍스트.
    *   GPT-4, GPT-5, LLaMA, ...
    *   현재 가장 활발히 연구되고 있는 LLM 분야.

# KoBART 모델로 문서 요약하기

In [1]:
import transformers  # HuggingFace의 transformers 기반 모델을 사용하기 위한 패키지

In [ ]:
# HuggingFace에서 오픈 소스로 공개된 모델을 다운로드해서 로컬에서 실행할 수 있도록 해줌.
pipe = transformers.pipeline(task="summarization",  # 문서 요약 작업
                             model="sshleifer/distilbart-cnn-12-6",
                             device=0)
# device=0: GPU 사용, device=-1(기본값): CPU

In [ ]:
type(pipe)  # Pipeline 클래스 객체

transformers.pipelines.text2text_generation.SummarizationPipeline

In [ ]:
sample_text = \
'''
During the 1905–06 English football season, New Brompton F.C. competed in the Southern League Division One. It was the 12th season in which the club competed in the Southern League and the 11th in Division One. The team began the season in poor form; they failed to score any goals in six of their first eight Southern League games. By the midpoint of the season, the team had won only three times and were close to the bottom of the league table. The team's form improved in the new year, with three wins in the first seven Southern League games of 1906, but they ended the season in similar fashion to how they had started it, failing to score in eight of the final nine league games. New Brompton finished the season in 17th place out of 18 teams in the division.

New Brompton also competed in the FA Cup, reaching the second round. The team played a total of 37 league and cup matches, winning 8, drawing 9 and losing 20. Bill Marriott was the club's top goalscorer, with four goals in the Southern League and one in the FA Cup; this figure was the lowest to date with which a player had finished a season as New Brompton's top scorer. Joe Walton made the most appearances, playing in 36 of the team's 37 competitive games. The highest attendance recorded at the club's home ground, Priestfield Road, was 5,500 for a game against Portsmouth on 27 January 1906.

The club's first match of the season, on 2 September, was away to Queens Park Rangers; Floyd, Phillips, Sheridan and Marriott all made their Southern League debuts for New Brompton.[13] Queens Park Rangers scored two goals in each half to win 4–0; the correspondent for The Daily News stated that the home team were "the better side at every point" and well deserved their large victory.[14] The first Southern League game of the season at New Brompton's ground, Priestfield Road, a week later against Bristol Rovers, resulted in a 3–0 win for the away team.[13][15] Phillips scored New Brompton's first Southern League goal of the season on 16 September and the team achieved their first victory, beating Northampton Town 2–0,[13][16] before losing again in their next game, a 4–0 defeat away to Portsmouth.[17] After a goalless draw with Swindon Town, New Brompton were 15th out of 18 teams in the Division One table at the end of September.[18]
'''

In [ ]:
# Pipeline 객체를 함수처럼 호출할 수 있음.
result = pipe(sample_text)

In [ ]:
type(result)  # dict를 원소로 갖는 리스트(list)

list

In [ ]:
# 리스트의 첫번째 원소 dict에서 키 summary_text에 해당하는 값을 출력
print(result[0]['summary_text'])

 During the 1905–06 English football season, New Brompton F.C. competed in the Southern League Division One . The team played 37 league and cup matches, winning 8, drawing 9 and losing 20 . The highest attendance recorded at the club's home ground, Priestfield Road, was 5,500 for a game against Portsmouth in January 1906 .


In [ ]:
sample_text_kor = \
'''
1896년 하계 올림픽(영어: 1896 Summer Olympics, Games of the I Olympiad, 그리스어: Θερινοί Ολυμπιακοί Αγώνες 1896)은 393년을 마지막으로 끝난 고대 올림픽 대회 이후 열린 첫 근대 올림픽 대회다. 1896년 4월 6일부터 4월 15일까지 그리스 아테네에서 개최되었다. 고대 그리스가 올림픽의 발상지여서 첫 근대 올림픽이 열리기에 적당한 장소였던 아테네는 1894년 6월 23일에 파리에서, 프랑스의 역사학자인 쿠베르탱이 주관한 올림픽 의회에서 만장일치로 개최지 자격을 얻었다. 또한 하계 올림픽이 진행되는 동안 국제 올림픽 위원회(IOC)가 조직되었다.

여러 어려움을 이겨낸 1회 올림픽은 성공적인 평가를 받았다. 당시의 국제 경기 중에서는 이 대회가 가장 많은 국제적인 참여를 이끌어냈다. 19세기 때 유일한 올림픽 경기장으로 쓰인 파나티나이코 경기장은 경기를 보려는 사람들이 몰려서 인산인해를 이루었다.[2] 개최국인 그리스에게 있어서 가장 절정이었던 부분은 마라톤 경기에서 자국 선수인 스피리돈 루이스가 승리의 영광을 차지한 것이었다. 이 대회에서는 카를 슈만이 레슬링과 체조에서 4개의 금메달을 따서 가장 많은 금메달을 획득했다. 가장 많이 메달을 딴 선수는 헤르만 바인게르트너로 6개의 메달을 땄다.

대회가 끝난 후 IOC는 이후의 올림픽을 계속 그리스에서 개최할 것인가를 놓고 피에르 드 쿠베르탱 남작과 그리스 임금인 요르요스 1세, 몇몇 미국 선수들 사이에서 갈등이 있었으나 1900년 대회가 이미 파리에서 열리기로 결정된 상태였고, 이후 올림픽은 세계를 순환하면서 개최하게 된다. 1906년 중간 올림픽을 제외하고는 그리스는 2004년 하계 올림픽을 개최할 때까지 108년간 올림픽을 개최하지 못했다.
'''

In [ ]:
# pipe(sample_text_kor)
# 해당 모델은 한글 텍스트로는 훈련이 되지 않아서 요약 작업을 실행할 수 없음.

In [ ]:
pipe_kor = transformers.pipeline(task='summarization',
                                 model='EbanLee/kobart-summary-v3')

In [ ]:
pipe_kor(sample_text_kor)

[{'summary_text': '1896년 하계 올림픽은 고대 올림픽 대회 이후 열린 첫 근대 올림픽 대회로 아테네에서 개최되었다.'}]

In [ ]:
# Pipeline 객체의 model 속성(property)의 config 속성(property)
print(pipe_kor.model.config)

BartConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "author": "EbanLee(rudwo6769@gmail.com)",
  "bos_token_id": 1,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.1,
  "d_model": 768,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 1,
  "do_blenderbot_90_layernorm": false,
  "dropout": 0.1,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 1,
  "extra_pos_embeddings": 2,
  "force_bos_token_to_be_generated": false,
  "forced_eos_token_id": 1,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "NEGATIVE",
    "1": "POSITIVE"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "kobart_version": 2.0,
  "label2id": {
    "NEGATIVE": 0

토크나이저(tokenizer): 텍스트를 토큰(정수)으로 변환하는 객체.

In [ ]:
print(pipe_kor.tokenizer.vocab_size)  # tokenizer가 사용하는 어휘 사전의 크기

30000


pipeline 함수는 LLM 모델과 텍스트를 토큰화하는 tokenizer 객체를 함께 다운로드함.

LLM 모델이 문서 요약을 하기 전에 입력 텍스트를 tokenizer를 사용해서 전처리를 하고, 모델이 예측을 수행.

In [ ]:
vocab = pipe_kor.tokenizer.vocab  # 어휘 사전 - dict(단어를 키로 하고 토큰을 값으로 하는)

In [ ]:
list(vocab.items())[:10]

[('쓇', 11647),
 ('어리', 27108),
 ('▁3번', 28391),
 ('둥', 9935),
 ('裹', 7373),
 ('歙', 4895),
 ('▁‘201', 22810),
 ('▁벌어진', 21092),
 ('▁재벌', 18954),
 ('블', 11011)]

## tokenizer 객체의 메서드

In [ ]:
tokenizer = pipe_kor.tokenizer

In [ ]:
# tokenize(): 텍스트 --> 토큰들의 리스트
tokens = tokenizer.tokenize('혼자 공부하는 머신러닝 딥러닝')

In [ ]:
print(tokens)

['▁혼자', '▁공부', '하는', '▁머', '신', '러', '닝', '▁', '딥', '러', '닝']


In [ ]:
# convert_tokens_to_ids(): 토큰들의 리스트를 토큰에 해당하는 정수들의 리스트로 변환
ids = tokenizer.convert_tokens_to_ids(tokens)

In [ ]:
print(ids)

[16814, 16962, 14049, 14771, 11467, 10277, 9747, 1700, 10021, 10277, 9747]


In [ ]:
# encode(): tokenize + convert_tokens_to_ids
encoded = tokenizer.encode('혼자 공부하는 머신러닝 딥러닝')

In [ ]:
print(encoded)

[0, 16814, 16962, 14049, 14771, 11467, 10277, 9747, 1700, 10021, 10277, 9747, 1]


In [ ]:
encoded2 = tokenizer.encode('KoBART 모델을 사용한 문서 요약')
print(encoded2)

[0, 14572, 310, 265, 264, 281, 283, 24224, 21032, 26052, 26200, 1]


In [ ]:
# convert_ids_to_tokens(): 토큰 아이디들의 리스트를 토큰들의 리스트로 변환
tokens2 = tokenizer.convert_ids_to_tokens(encoded2)
print(tokens2)

['<s>', '▁K', 'o', 'B', 'A', 'R', 'T', '▁모델을', '▁사용한', '▁문서', '▁요약', '</s>']


In [ ]:
# decode(): 토큰 아이디들을 갖는 리스트를 텍스트로 변환
tokenizer.decode(encoded2)

'<s> KoBART 모델을 사용한 문서 요약</s>'

In [ ]:
tokenizer.decode(encoded2[1:-1])

'KoBART 모델을 사용한 문서 요약'

Pipeline 동작 원리

텍스트 입력 --> tokenizer.encode() --> LLM 모델 --> tokenizer.decode()

# EXAONE-3.5 모델로 텍스트 생성하기

In [ ]:
# EXAONE-3.5 모델에서 사용하는 토크나이저 객체를 로딩
exaone_tokenizer = \
    transformers.AutoTokenizer.from_pretrained('LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct')

In [ ]:
# EXAONE-3.5 Pipeline 객체 생성
exaone = transformers.pipeline(task='text-generation',
                               model='LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct',
                               tokenizer=exaone_tokenizer,
                               device=0,
                               trust_remote_code=True)

In [5]:
# 메시지 템플릿: 'role'과 'content' 키를 갖는 dict를 원소로 갖는 리스트(list)
# role: 역할(system, user, assistant)
# content: 내용
messages = [
    {
        'role': 'system',
        'content': '''너는 쇼핑몰 홈페이지에 올라온 Q&A에 답변을 하는 챗봇이야.
                확정적인 답변을 하지 말고 제품 담당자가 정확한 답변을 하기 위해 시간이
                필요하다는 간단하고 친절한 답변을 생성해줘.'''
    },
    {
        'role': 'user',
        'content': '이 다이어리에는 내년도 공휴일 정보가 있나요?'
    }
]

In [7]:
result = exaone(messages, max_new_tokens=200)

[{'generated_text': [{'role': 'system', 'content': '너는 쇼핑몰 홈페이지에 올라온 Q&A에 답변을 하는 챗봇이야.\n                확정적인 답변을 하지 말고 제품 담당자가 정확한 답변을 하기 위해 시간이\n                필요하다는 간단하고 친절한 답변을 생성해줘.'}, {'role': 'user', 'content': '이 다이어리에는 내년도 공휴일 정보가 있나요?'}, {'role': 'assistant', 'content': ' 다이어리에 내년의 정확한 공휴일 정보가 포함되어 있는지 확인하려면, 먼저 현재 다이어리의 내용물을 직접 확인하시는 것이 가장 좋을 것 같습니다. 공휴일 정보는 국가마다 다르고 연도별로도 변동이 있을 수 있으니까요. 만약 직접 확인이 어려우시다면, 고객 서비스 팀에 연락하시거나 온라인으로 문의하시면 더 정확하고 최신의 정보를 제공받으실 수 있을 것 같습니다. 도움이 되셨으면 좋겠습니다!'}]}]


In [8]:
result  #> 리턴 값 - dict 1개를 원소로 갖는 list
# dict는 'generated_text' 키와 값 1개로만 이루어진 딕셔너리.
# generated_text의 값(value)은 메시지 프롬프트(dict)들을 가지고 있는 리스트(list).
# generated_text의 가장 마지막 원소가 모델이 생성한 답변.

[{'generated_text': [{'role': 'system',
    'content': '너는 쇼핑몰 홈페이지에 올라온 Q&A에 답변을 하는 챗봇이야.\n                확정적인 답변을 하지 말고 제품 담당자가 정확한 답변을 하기 위해 시간이\n                필요하다는 간단하고 친절한 답변을 생성해줘.'},
   {'role': 'user', 'content': '이 다이어리에는 내년도 공휴일 정보가 있나요?'},
   {'role': 'assistant',
    'content': ' 다이어리에 내년의 정확한 공휴일 정보가 포함되어 있는지 확인하려면, 먼저 현재 다이어리의 내용물을 직접 확인하시는 것이 가장 좋을 것 같습니다. 공휴일 정보는 국가마다 다르고 연도별로도 변동이 있을 수 있으니까요. 만약 직접 확인이 어려우시다면, 고객 서비스 팀에 연락하시거나 온라인으로 문의하시면 더 정확하고 최신의 정보를 제공받으실 수 있을 것 같습니다. 도움이 되셨으면 좋겠습니다!'}]}]

In [13]:
result = exaone(messages, max_new_tokens=200, return_full_text=False)
#> return_full_text=True(기본값): 메시지 프롬프트의 이력이 모두 generated_text의 값으로 리턴.
#> return_full_text=False: generated_text의 값은 모델이 생성한 답변(텍스트)만 포함.

In [14]:
result

[{'generated_text': '죄송합니다만, 현재로선 다이어리의 내년 공휴일 정보를 직접 확인할 수 있는 기능이 없어요. 정확한 내용을 확인하려면 제품 담당자에게 문의하시는 것이 가장 좋을 것 같습니다. 그분께서는 즉시 최신 정보를 제공해 드릴 수 있을 거예요. 도움이 필요하시다면 알려주세요! 😊'}]

In [15]:
result[0]['generated_text']

'죄송합니다만, 현재로선 다이어리의 내년 공휴일 정보를 직접 확인할 수 있는 기능이 없어요. 정확한 내용을 확인하려면 제품 담당자에게 문의하시는 것이 가장 좋을 것 같습니다. 그분께서는 즉시 최신 정보를 제공해 드릴 수 있을 거예요. 도움이 필요하시다면 알려주세요! 😊'

텍스트를 생성할 때마다 토큰들에서 샘플링을 하기 때문에 모델을 실행할 때마다 다른 결과를 줌.

In [20]:
result = exaone(messages, max_new_tokens=200, return_full_text=False, do_sample=False)

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [21]:
result[0]['generated_text']

'안녕하세요! 다이어리에 내년의 공휴일 정보가 포함되어 있는지 확인해 드리기 위해 잠시 시간을 내주실 수 있을까요? 제품 담당자께서 자세히 확인해 주시고 정확한 답변을 드릴 수 있도록 하겠습니다. 조금만 기다려 주세요! 😊'

`do_sample` 파라미터: 샘플링 전략을 사용할 지 말 지를 설정하는 파라미터. 기본값은 True.

*   `do_sample=True`: 메시지 프롬프트가 같아도 실행할 때마다 생성되는 텍스트가 달라짐.
*   `do_sample=False`:
    *   가장 높은 확률을 갖는 토큰만 선택해서 텍스트를 생성.
    *   메시지 프롬프트가 같으면 항상 생성되는 텍스트가 같음.